In [0]:
! pip install bs4
! pip install contractions
! pip install python-box

     |████████████████████████████████| 245kB 4.0MB/s 
     |████████████████████████████████| 317kB 41.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81709 sha256=423037c36cf004feb1bc13d02ddb95edc510153821eca2e3f321136801a47a60
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
import string
import contractions
import nltk
from nltk.tokenize import TweetTokenizer
import os
import re

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Step 1: Take a look at the data

In [0]:
dir = "/content/drive/My Drive/ML-data/tweetsense/sentiment140/"

In [0]:
col_names = ['label', 'id', 'date', 'flag', 'user', 'text']
df = pd.read_csv(dir + '/training.csv', encoding='latin-1', names=col_names)
df = df[['text', 'label']]
print(df.head(2))
print("\nThere are {} tweets in total.\n".format(len(df)))
print("There are {} positve tweets".format(np.sum(df["label"] == 4)))
print("There are {} neutral tweets".format(np.sum(df["label"] == 2)))
print("There are {} negative tweets".format(np.sum(df["label"] == 0)))

                                                text  label
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...      0
1  is upset that he can't update his Facebook by ...      0

There are 1600000 tweets in total.

There are 800000 positve tweets
There are 0 neutral tweets
There are 800000 negative tweets


### Step 2: Define Preprocess steps

In [0]:
def remove_html(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    return souped

text = "&amp; I like it"
remove_html(text)

'& I like it'

In [0]:
def remove_url(text):
    return re.sub(r'http\S+', '', text)

text = "Check this video http://bit.ly/IMXUM"
remove_url(text)

'Check this video '

In [0]:
import re
def remove_mention(text):
    return re.sub('@[^\s]+','',text)

text = "@David hi"
remove_mention(text)

' hi'

In [0]:
def remove_hashtag(text):
    return re.sub(r'#\w*', '', text)

text = "#trumpisidiot hahah"
remove_hashtag(text)

' hahah'

In [0]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

text = 'string with "punctuation" inside of it! Does this work? I hope so.'
remove_punctuation(text)

'string with punctuation inside of it Does this work I hope so'

In [0]:
def fix_contractions(text):
    return contractions.fix(text)

text = "I can't help you!"
fix_contractions(text)

'I can not help you!'

In [0]:
def preprocess(text):
    text = remove_html(text)
    text = remove_mention(text)
    text = remove_url(text)
    text = remove_hashtag(text)
    text = fix_contractions(text)

    words = TweetTokenizer().tokenize(text)
    words = [word.lower() for word in words]
    # Remove punctuation
    words = [remove_punctuation(word) for word in words]
    # Remove words that contain numeric values
    words = [word for word in words if word.isalpha()]
    
    return " ".join(words)

text = """
boy's cars
I can't help you. #Fucker. 
Check this out http://shit_url
Lebron..sh*t was hilarious...LMAO!!!
"""
preprocess(text)

'boys cars i can not help you check this out lebron sh t was hilarious lmao'

### Step 3: Preprocess training, val and test data

In [0]:
if not os.path.isfile("/content/drive/My Drive/ML-data/tweetsense/preprocessed_data/train.csv"):
    index = 0
    for i, row in df.iterrows():
        if index % 1000 == 0:
            print("Finished processing {} training data".format(index))
        index += 1
        text = df.at[i,'text']
        text = preprocess(text)
        df.at[i,'text'] = text
else:
    df = pd.read_csv(
        "/content/drive/My Drive/ML-data/tweetsense/preprocessed_data/train.csv", 
        encoding='latin-1'
    )
    df = df.dropna(axis = 0, how ='any')

In [0]:
df.head(5)

,text,label
0,awww that is a bummer you shoulda got david ca...,0
1,is upset that he can not update his facebook b...,0
2,i dived many times for the ball managed to sav...,0
3,my whole body feels itchy and like its on fire,0
4,no it is not behaving at all i am mad why am i...,0


In [0]:
train, val = train_test_split(df, test_size=0.3)
print("Training set size: {}".format(len(train)))
print("Validation set size: {}".format(len(val)))

Training set size: 1117502
Validation set size: 478930


In [0]:
train.to_csv('/content/drive/My Drive/ML-data/tweetsense/data/train.csv', index=False)
val.to_csv('/content/drive/My Drive/ML-data/tweetsense/data/val.csv', index=False)

### Step 4: Set up Bert Dependency

In [0]:
! nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
!python --version

Python 3.6.8


In [0]:
import torch
print(torch.__version__)

1.3.1


In [0]:
! pip install pytorch-pretrained-bert
! pip install fast_bert
! pip install pytorch-transformers

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 645kB 8.8MB/s 
     |████████████████████████████████| 71kB 2.3MB/s 
     |████████████████████████████████| 368kB 7.9MB/s 
     |████████████████████████████████| 194kB 43.4MB/s 
     |████████████████████████████████| 860kB 46.6MB/s 
     |████████████████████████████████| 1.0MB 47.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=e676118a575c5f234c5849b8dc7c2e043132b4356cc72bc1a0abf12c5710ef1f
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses
     |████████████████████████████████| 184kB 2.8MB/s 


In [0]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

fatal: destination path 'apex' already exists and is not an empty directory.
/content/apex
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-gc3q2fwy
Created temporary directory: /tmp/pip-req-tracker-2h3jutje
Created requirements tracker '/tmp/pip-req-tracker-2h3jutje'
Created temporary directory: /tmp/pip-install-0qykgzqx
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-4j4sbdd1
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-2h3jutje'
    Running setup.py (path:/tmp/pip-req-build-4j4sbdd1/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.3.1
    running egg_info
    creating /tmp/pip-req-build-4j4sbdd1/pip-egg-info/a

### Step 5: Set up Bert Model

In [0]:
import torch
import apex
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
from box import Box

In [0]:
args = Box({
    "model_name": 'bert-base-uncased',
    "model_type": 'bert',
    "max_seq_length": 128,
    "train_batch_size": 32,
    "learning_rate": 6e-5,
    "num_train_epochs": 6,
    "warmup_steps": 500,
    "fp16": True,
    "logging_steps": 50,
    "multi_gpu": True if torch.cuda.device_count() > 1 else False
})

In [0]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S'
)

logger = logging.getLogger()
logger.info(args)

12/01/2019 06:29:00 - INFO - root -   {'model_name': 'bert-base-uncased', 'model_type': 'bert', 'max_seq_length': 128, 'train_batch_size': 32, 'learning_rate': 6e-05, 'num_train_epochs': 6, 'warmup_steps': 500, 'fp16': True, 'logging_steps': 50, 'multi_gpu': False}


In [0]:
from pathlib import Path
DATA_PATH = Path('/content/drive/My Drive/ML-data/tweetsense/data/')     # path for data files (train and val)
LABEL_PATH = Path('/content/drive/My Drive/ML-data/tweetsense/labels/')  # path for labels file
MODEL_PATH=Path('/content/drive/My Drive/ML-data/tweetsense/models/')    # path for model artifacts to be stored
LOG_PATH=Path('/content/drive/My Drive/ML-data/tweetsense/logs/')       # path for log files to be stored
print(DATA_PATH)

/content/drive/My Drive/ML-data/tweetsense/data


In [0]:
databunch = BertDataBunch(
    DATA_PATH, 
    LABEL_PATH, 
    args.model_name,
    train_file='train.csv', 
    val_file='val.csv', 
    label_file='labels.csv',
    text_col='text', 
    label_col='label',
    batch_size_per_gpu=args.train_batch_size, 
    max_seq_length=args.max_seq_length, 
    multi_gpu=args.multi_gpu, 
    multi_label=False,
    model_type=args.model_type
)

12/01/2019 06:29:02 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
12/01/2019 06:29:02 - INFO - root -   Loading features from cached file /content/drive/My Drive/ML-data/tweetsense/data/cache/cached_bert_train_multi_class_128_train.csv
12/01/2019 06:29:52 - INFO - root -   Loading features from cached file /content/drive/My Drive/ML-data/tweetsense/data/cache/cached_bert_dev_multi_class_128_val.csv


In [0]:
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

# The learner contains the logic for training loop, validation loop, 
# optimiser strategies and key metrics calculation
learner = BertLearner.from_pretrained_model(
    databunch, 
    pretrained_path=args.model_name, 
    metrics=metrics, 
    device=device_cuda, 
    logger=logger, 
    output_dir="/content/drive/My Drive/ML-data/tweetsense/output",
    finetuned_wgts_path=None, 
    warmup_steps=args.warmup_steps,
    is_fp16=args.fp16,
    multi_gpu=args.multi_gpu, 
    multi_label=False,
    logging_steps=args.logging_steps
)

12/01/2019 06:30:11 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
12/01/2019 06:30:11 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

12/

### Step 6: Train Bert

In [0]:
learner.fit(
    epochs=1,
	lr=6e-5,
	validate=False, 	# Evaluate the model after each epoch
	schedule_type="warmup_cosine",
    optimizer_type="lamb"
)

12/01/2019 06:47:47 - INFO - root -   ***** Running training *****
12/01/2019 06:47:47 - INFO - root -     Num examples = 1117502
12/01/2019 06:47:47 - INFO - root -     Num Epochs = 1
12/01/2019 06:47:47 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 32
12/01/2019 06:47:47 - INFO - root -     Gradient Accumulation steps = 1
12/01/2019 06:47:47 - INFO - root -     Total optimization steps = 34922


/content/drive/My Drive/ML-data/tweetsense/output/tensorboard
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


12/01/2019 06:48:16 - INFO - root -   lr after step 50: 6e-06
12/01/2019 06:48:16 - INFO - root -   train_loss after step 50: 0.7025802683830261
12/01/2019 06:48:44 - INFO - root -   lr after step 100: 1.2e-05
12/01/2019 06:48:44 - INFO - root -   train_loss after step 100: 0.6995965993404388
12/01/2019 06:49:12 - INFO - root -   lr after step 150: 1.8e-05
12/01/2019 06:49:12 - INFO - root -   train_loss after step 150: 0.6899114167690277
12/01/2019 06:49:40 - INFO - root -   lr after step 200: 2.4e-05
12/01/2019 06:49:40 - INFO - root -   train_loss after step 200: 0.6824549758434295
12/01/2019 06:50:08 - INFO - root -   lr after step 250: 3e-05
12/01/2019 06:50:08 - INFO - root -   train_loss after step 250: 0.671521338224411
12/01/2019 06:50:36 - INFO - root -   lr after step 300: 3.6e-05
12/01/2019 06:50:36 - INFO - root -   train_loss after step 300: 0.6529994702339172
12/01/2019 06:51:05 - INFO - root -   lr after step 350: 4.2e-05
12/01/2019 06:51:05 - INFO - root -   train_loss

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


12/01/2019 06:51:33 - INFO - root -   lr after step 400: 4.8e-05
12/01/2019 06:51:33 - INFO - root -   train_loss after step 400: 0.6191174590587616
12/01/2019 06:52:01 - INFO - root -   lr after step 450: 5.4000000000000005e-05
12/01/2019 06:52:01 - INFO - root -   train_loss after step 450: 0.5952912652492524
12/01/2019 06:52:29 - INFO - root -   lr after step 500: 6e-05
12/01/2019 06:52:29 - INFO - root -   train_loss after step 500: 0.5581774938106537


...

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


12/01/2019 11:48:03 - INFO - root -   lr after step 31900: 1.1338429188993748e-06
12/01/2019 11:48:03 - INFO - root -   train_loss after step 31900: 0.3423261530697346
12/01/2019 11:48:32 - INFO - root -   lr after step 31950: 1.0968621313584192e-06
12/01/2019 11:48:32 - INFO - root -   train_loss after step 31950: 0.3482692486047745
12/01/2019 11:49:00 - INFO - root -   lr after step 32000: 1.0604832264257801e-06
12/01/2019 11:49:00 - INFO - root -   train_loss after step 32000: 0.3530027735233307
12/01/2019 11:49:28 - INFO - root -   lr after step 32050: 1.0247069616603933e-06
12/01/2019 11:49:28 - INFO - root -   train_loss after step 32050: 0.35443165063858034
12/01/2019 11:49:56 - INFO - root -   lr after step 32100: 9.895340820717358e-07
12/01/2019 11:49:56 - INFO - root -   train_loss after step 32100: 0.3394032555818558
12/01/2019 11:50:24 - INFO - root -   lr after step 32150: 9.549653201043063e-07
12/01/2019 11:50:24 - INFO - root -   train_loss after step 32150: 0.3398402583

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


12/01/2019 12:12:01 - INFO - root -   lr after step 34450: 2.7831437577813256e-08
12/01/2019 12:12:01 - INFO - root -   train_loss after step 34450: 0.38236106097698214
12/01/2019 12:12:29 - INFO - root -   lr after step 34500: 2.2247951026148628e-08
12/01/2019 12:12:29 - INFO - root -   train_loss after step 34500: 0.34557484537363053
12/01/2019 12:12:57 - INFO - root -   lr after step 34550: 1.7288724984775296e-08
12/01/2019 12:12:57 - INFO - root -   train_loss after step 34550: 0.34328680276870727
12/01/2019 12:13:25 - INFO - root -   lr after step 34600: 1.2953862725241238e-08
12/01/2019 12:13:25 - INFO - root -   train_loss after step 34600: 0.3669495138525963
12/01/2019 12:13:53 - INFO - root -   lr after step 34650: 9.243454517274818e-09
12/01/2019 12:13:53 - INFO - root -   train_loss after step 34650: 0.3542681756615639
12/01/2019 12:14:21 - INFO - root -   lr after step 34700: 6.157577626879629e-09
12/01/2019 12:14:21 - INFO - root -   train_loss after step 34700: 0.31473133

(34922, 0.3752534805951345)

In [0]:
# Save the model into a file
learner.save_model()

12/01/2019 12:16:28 - INFO - transformers.configuration_utils -   Configuration saved in /content/drive/My Drive/ML-data/tweetsense/output/model_out/config.json
12/01/2019 12:16:29 - INFO - transformers.modeling_utils -   Model weights saved in /content/drive/My Drive/ML-data/tweetsense/output/model_out/pytorch_model.bin


### Step 7: Inference

In [0]:
from fast_bert.prediction import BertClassificationPredictor

MODEL_PATH = "/content/drive/My Drive/ML-data/tweetsense/output/model_out"

predictor = BertClassificationPredictor(
				model_path=MODEL_PATH,
				label_path=LABEL_PATH, # location for labels.csv file
				multi_label=False,
				model_type='bert',
				do_lower_case=False)

def predict_sentiment(tweet):
    prediction = predictor.predict(tweet)
    
    # if prediction[0][1] < 0.65 and prediction[1][1] < 0.65:
    #     # print("Neutral")
    #     return 2
    # else:
    result = max(prediction, key=lambda x: x[1])
    # if result[0] == "0":
        # print("Negative")
    # elif result[0] == "4":
        # print("Positive")

    return result[0]

In [0]:
predict_sentiment("just get me result for this text")

'4'

In [0]:
predict_sentiment("I hate you")

Negative


'0'

In [0]:
predict_sentiment("fuck this shit")

Negative


'0'

In [0]:
predict_sentiment("I love you")

Positive


'4'

In [0]:
predict_sentiment("This is stupid")

Negative


'0'

### Step 8: Evaluation

In [0]:
dir = "/content/drive/My Drive/ML-data/tweetsense/sentiment140/"

col_names = ['label', 'id', 'date', 'flag', 'user', 'text']
df = pd.read_csv(dir + '/test.csv', encoding='latin-1', names=col_names)
df = df[['text', 'label']]
print(df.head(2))
print("\nThere are {} tweets in total.\n".format(len(df)))
print("There are {} positve tweets".format(np.sum(df["label"] == 4)))
print("There are {} neutral tweets".format(np.sum(df["label"] == 2)))
print("There are {} negative tweets".format(np.sum(df["label"] == 0)))

                                                text  label
0  @stellargirl I loooooooovvvvvveee my Kindle2. ...      4
1  Reading my kindle2...  Love it... Lee childs i...      4

There are 498 tweets in total.

There are 182 positve tweets
There are 139 neutral tweets
There are 177 negative tweets


In [0]:
df = df[df['label'] != 2]
y_true = df['label'].values.tolist()
X_test = df['text'].values.tolist()
print(len(X_test))
print(len(y_true))

359
359


In [0]:
y_pred = []

for x in X_test:
    x = preprocess(x)

    if x == "":
        print("Empty string. Assume positive")
        y_pred.append(4)

    predicted_label = predict_sentiment(x)
    y_pred.append(int(predicted_label))

In [0]:
print(y_true)
print(y_pred)

[4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 0, 4, 0, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 4, 0, 4, 4, 4, 0, 4, 4, 0, 0, 4, 4, 4, 0, 4, 4, 4, 0, 0, 4, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 4, 0, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 4, 4, 4, 4, 4, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 4, 0, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 4, 0, 0, 4, 4, 4, 0, 4, 4, 4, 4, 0, 4, 0, 4, 0, 0, 0, 0, 

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [0]:
accuracy_score(y_true, y_pred)


0.8328690807799443

In [0]:
f1_score(y_true, y_pred, average='weighted') 

0.8328223671689822